In [1]:
import os
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy

from sklearn import preprocessing, pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from isotree import IsolationForest
from sklearn import tree, ensemble
from cfmining.mip_algorithms import ForestRecourseActions
from cfmining.predictors import TreeClassifier

from cfmining.mip_algorithms import LinearRecourseActions, LinearRecourseActionsMulti
from cfmining.algorithms import MAPOCAM, BruteForce, Greedy, MAPOCAM2
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import MonotoneClassifier, GeneralClassifier, TreeClassifier, LinearClassifier, LinearRule, GeneralClassifier_Shap
from cfmining.action_set import ActionSet

from results import save_result

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

## Preparing dataset

In [3]:
data_dir = "data/"
data_name = 'german'
data_file = os.path.join(data_dir, '%s_processed.csv' % data_name)
## load and process data
german_df = pd.read_csv(data_file).reset_index(drop=True)
german_df = (german_df
             .assign(isMale=lambda df: (df['Gender']=='Male').astype(int))
             .drop(['PurposeOfLoan', 'Gender', 'OtherLoansAtStore'], axis=1)
            )

y = german_df['GoodCustomer']
y = (y == 1).astype(int) # tranform y to [0, 1]
X = german_df.drop('GoodCustomer', axis=1)
#X = X[['ForeignWorker', 'Single', 'Age', 'LoanDuration', 'LoanAmount', 'LoanRateAsPercentOfIncome', 'isMale', 'YearsAtCurrentHome',
#'NumberOfOtherLoansAtBank', 'NumberOfLiableIndividuals', 'HasTelephone',]]

In [4]:
Xtr, Xts, ytr, yts = train_test_split(X, y, test_size=100)

In [5]:
#outlier_detection = IsolationForest(contamination=0.1, n_estimators=50)
outlier_detection = IsolationForest(ndim=1, sample_size=256, max_depth=8, ntrees=100, missing_action="divide")
outlier_detection.fit(Xtr);
print(np.unique(outlier_detection.predict(Xtr) > 0.6, return_counts=True))

(array([False,  True]), array([889,  11]))


## Models

In [6]:
stand = preprocessing.StandardScaler()
clf_base = LogisticRegression(max_iter=1000, class_weight='balanced')
clf = pipeline.Pipeline([('std', stand), ('clf', clf_base)])

grid = GridSearchCV(
  clf, param_grid={'clf__C': np.logspace(-4, 3)},
  cv=5,
  scoring='roc_auc',
  verbose=0
)

grid.fit(Xtr, ytr)
clf_lgr = grid.best_estimator_

clf_rt = ensemble.RandomForestClassifier(
  n_estimators=10, 
  max_depth=5, 
  max_leaf_nodes=31,
  class_weight='balanced_subsample'
)
clf_rt.fit(Xtr, ytr);

In [7]:
threshold=0.5
scores_lgr = pd.Series(clf_lgr.predict_proba(Xts)[:, 1])
denied_individuals_lgr = scores_lgr.loc[lambda s: s < threshold].index

scores_rt = pd.Series(clf_rt.predict_proba(Xts)[:, 1])
denied_individuals_rt = scores_rt.loc[lambda s: s < threshold].index

## Setting ActionSet base parameters

In [8]:
action_set_base = ActionSet(X = X)
for feat in action_set_base:
    if feat.name in ['Age', 'JobClassIsSkilled', 'OwnsHouse', 'isMale', 'JobClassIsSkilled']:
        feat.mutable = False
    if feat.name in ['Single', 'ForeignWorker', 'RentsHouse']:
        feat.mutable = False
    if feat.name in ['LoanDuration', 'LoanAmount', 'LoanRateAsPercentOfIncome', 'NumberOfOtherLoansAtBank', 'MissedPayments',
                     'CriticalAccountOrLoansElsewhere', 'OtherLoansAtBank', 'Unemployed', 'YearsAtCurrentJob_lt_1']:
        feat.mutable = True
    if feat.name in ['YearsAtCurrentHome', 'NumberOfLiableIndividuals', 'HasTelephone', 'CheckingAccountBalance_geq_0',
                     'CheckingAccountBalance_geq_200', 'SavingsAccountBalance_geq_100', 'SavingsAccountBalance_geq_500',
                     'NoCurrentLoan', 'HasCoapplicant', 'HasGuarantor', 'YearsAtCurrentJob_geq_4']:
        feat.mutable = True
    
    if feat.name == "LoanDuration":
        feat.step_type = "absolute"
        feat.step_size = 6
    
    if feat.name == "LoanAmount":
        feat.step_size = 0.1

    feat.step_direction = 0
    feat.update_grid()

In [10]:
# action set that uses the monoticity of the classifier
action_set_lgr = copy.deepcopy(action_set_base)
action_set_lgr.embed_linear_clf(clf_lgr[1])

# action set that uses the splits of the trees
action_set_tree = copy.deepcopy(action_set_base)
action_set_tree.embed_forest(clf_rt)
for action in action_set_tree:
    action.flip_direction = action.step_direction

percCalc_lgr = PercentileCalculator(action_set=action_set_lgr)
percCalc_rt = PercentileCalculator(action_set=action_set_tree)

## Counterfactuals Logistic

In [11]:
clf_lgr_mapocam = MonotoneClassifier(clf_lgr, X=Xtr, y=ytr, threshold=threshold)
clf_lgr_shap = GeneralClassifier_Shap(clf_lgr, X=Xtr, y=ytr, threshold=threshold)

PermutationExplainer explainer: 901it [00:48, 17.54it/s]                         


In [13]:
solutions = {
    "mapocam" : [],
    "mapocam_2" : [],
    "mapocam_2_outlier" : [],
    "mapocam_2_outlier_v2" : [],
}
computing_time = {
    "mapocam" : [],
    "mapocam_2" : [],
    "mapocam_2_outlier" : [],
    "mapocam_2_outlier_v2" : [],  
}

# save the solutions for all individuals (and all algorithms)
for i in denied_individuals_lgr:
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc_lgr)

    start = time.perf_counter()
    mapocam = MAPOCAM(
        action_set_lgr, 
        individual, 
        clf_lgr_mapocam,
        max_changes=float('inf'), 
        compare=criteria
    )
    mapocam.fit()
    computing_time["mapocam"].append(time.perf_counter() - start)
    solutions["mapocam"].append(mapocam.solutions)

    start = time.perf_counter()
    mapocam = MAPOCAM2(
        action_set_base, 
        individual, 
        clf_lgr_shap,
        max_changes=float('inf'), 
        compare=criteria
    )
    mapocam.fit()
    computing_time["mapocam_2"].append(time.perf_counter() - start)
    solutions["mapocam_2"].append(mapocam.solutions)

    start = time.perf_counter()
    mapocam = MAPOCAM2(
        action_set_base, 
        individual, 
        clf_lgr_shap, 
        outlier_detection,
        max_changes=float('inf'), 
        compare=criteria
    )
    mapocam.fit()
    computing_time["mapocam_2_outlier"].append(time.perf_counter() - start)
    solutions["mapocam_2_outlier"].append(mapocam.solutions)

    start = time.perf_counter()
    mapocam = MAPOCAM2(
        action_set_base, 
        individual, 
        clf_lgr_shap, 
        outlier_detection,
        estimate_outlier=True,
        max_changes=float('inf'), 
        compare=criteria
    )
    mapocam.fit()
    computing_time["mapocam_2_outlier_v2"].append(time.perf_counter() - start)
    solutions["mapocam_2_outlier_v2"].append(mapocam.solutions)

X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScaler was fitted with feature names
X does not have valid feature names, but StandardScale

In [16]:
for key, value in computing_time.items():
    print(f"{key} {np.mean(value):.4f} +- {np.std(value):.4f}")

mapocam 0.0445 +- 0.0385
mapocam_2 0.8661 +- 2.4873
mapocam_2_outlier 0.8017 +- 2.1156
mapocam_2_outlier_v2 0.6224 +- 1.3927


In [43]:
def check_equal_solutions(solutions_1, solutions_2):
    if len(solutions_1) != len(solutions_2):
        return False
    
    for sol_1 in solutions_1:
        for sol_2 in solutions_2:
            if not np.all(sol_1 == sol_2):
                return False
            
    return True

def table_with_solutions(model, individual, solutions_1, solutions_2, solutions_1_name = "sol1", solutions_2_name = "sol2", show_fixed_columns = True):
    prob = model.predict_proba(pd.DataFrame(individual).T)[0][1]
    individual_prob = individual._append(pd.Series({"prob": prob}))
    table = pd.DataFrame(individual_prob)
    table.columns = ['original']

    if len(solutions_1) == 0:
        table[solutions_1_name + ' (0)'] = None
    for i, sol in enumerate(solutions_1):
        sol_df = pd.DataFrame(sol.reshape(1, -1))
        sol_df.columns = individual.index
        prob = model.predict_proba(sol_df)[0][1]
        table[f"{solutions_1_name} ({i})"] = np.concatenate((sol, [prob]))
    
    if len(solutions_2) == 0:
        table[solutions_2_name + ' (0)'] = None
    for i, sol in enumerate(solutions_2):
        sol_df = pd.DataFrame(sol.reshape(1, -1))
        sol_df.columns = individual.index
        prob = model.predict_proba(sol_df)[0][1]
        table[f"{solutions_2_name} ({i})"] = np.concatenate((sol, [prob]))

    table_T = table.T
    changed_columns = []
    fixed_columns = []
    for col in table_T.columns:
        # get unique that are not none
        unique = table_T[col].unique()
        unique = unique[~pd.isnull(unique)]
        if len(unique) == 1:
            fixed_columns.append(col)
        else:
            changed_columns.append(col)

    #table_T = table_T[changed_columns + fixed_columns]
    if show_fixed_columns:
        table_T = table_T[changed_columns + fixed_columns]
    else:
        table_T = table_T[changed_columns]
    table = table_T.T

    return table

In [44]:
for i in range(len(denied_individuals_lgr)):
    sol1 = solutions["mapocam"][i]
    sol2 = solutions["mapocam_2"][i]
        
    if check_equal_solutions(sol1, sol2):
        continue

    individual = Xts.iloc[denied_individuals_lgr[i]]
    table = table_with_solutions(clf_lgr, individual, sol1, sol2, "mapocam1", "mapocam2", False)
    print(table)
    print("---")


                                    original  mapocam1 (0)  mapocam2 (0)
LoanAmount                       1919.000000    1801.00000   1919.000000
CriticalAccountOrLoansElsewhere     0.000000       0.00000      1.000000
HasGuarantor                        0.000000       1.00000      0.000000
prob                                0.400952       0.51273      0.545781
---
               original  mapocam1 (0)  mapocam2 (0)
LoanDuration  11.000000     11.000000      6.000000
HasGuarantor   0.000000      1.000000      0.000000
prob           0.496884      0.606771      0.521286
---
                                    original  mapocam1 (0)  mapocam2 (0)
LoanDuration                       48.000000     42.000000       6.00000
LoanAmount                       8487.000000   7305.000000    8487.00000
CheckingAccountBalance_geq_200      0.000000      1.000000       0.00000
CriticalAccountOrLoansElsewhere     0.000000      0.000000       1.00000
HasGuarantor                        0.000000      1.00

In [46]:
for i in range(len(denied_individuals_lgr)):
    sol1 = solutions["mapocam_2"][i]
    sol2 = solutions["mapocam_2_outlier"][i]
        
    if check_equal_solutions(sol1, sol2):
        continue

    individual = Xts.iloc[denied_individuals_lgr[i]]
    table = table_with_solutions(clf_lgr, individual, sol1, sol2, "mapocam2", "mapocam2_outlier", False)
    print(table)
    print("---")


                                 original  mapocam2 (0)  mapocam2_outlier (0)
LoanRateAsPercentOfIncome        3.000000      3.000000              2.000000
CheckingAccountBalance_geq_0     1.000000      1.000000              0.000000
CriticalAccountOrLoansElsewhere  0.000000      1.000000              0.000000
prob                             0.399955      0.544752              0.511407
---


In [47]:
for i in range(len(denied_individuals_lgr)):
    sol1 = solutions["mapocam_2"][i]
    sol2 = solutions["mapocam_2_outlier_v2"][i]
        
    if check_equal_solutions(sol1, sol2):
        continue

    individual = Xts.iloc[denied_individuals_lgr[i]]
    table = table_with_solutions(clf_lgr, individual, sol1, sol2, "mapocam2_outlier", "mapocam2_outlier_v2", False)
    print(table)
    print("---")


                                 original  mapocam2_outlier (0)  \
LoanRateAsPercentOfIncome        3.000000              3.000000   
CheckingAccountBalance_geq_0     1.000000              1.000000   
CriticalAccountOrLoansElsewhere  0.000000              1.000000   
prob                             0.399955              0.544752   

                                 mapocam2_outlier_v2 (0)  
LoanRateAsPercentOfIncome                       2.000000  
CheckingAccountBalance_geq_0                    0.000000  
CriticalAccountOrLoansElsewhere                 0.000000  
prob                                            0.511407  
---


## Counterfactuals RF

In [48]:
clf_rf_mapocam =  TreeClassifier(clf_rt, Xtr, ytr, threshold=threshold, use_predict_max=True)
clf_rf_shap = GeneralClassifier_Shap(clf_rt, X=Xtr, y=ytr, threshold=threshold)

PermutationExplainer explainer: 901it [00:34, 18.82it/s]                         


In [50]:
solutions = {
    "mapocam" : [],
    "mapocam_2" : [],
    "mapocam_2_outlier" : [],
    "mapocam_2_outlier_v2" : [],
}
for i in denied_individuals_rt:
    individual = Xts.iloc[i].values
    criteria = PercentileCriterion(individual, percCalc_rt)
    clf_rf_mapocam.fit(individual, action_set_tree)
    mapocam = MAPOCAM(
        action_set_tree, 
        individual, 
        clf_rf_mapocam,
        max_changes=float('inf'), 
        compare=criteria
    )
    mapocam.fit()
    solutions["mapocam"].append(mapocam.solutions)

    mapocam = MAPOCAM2(
        action_set_base, 
        individual,
        clf_rf_shap, 
        #outlier_detection,
        max_changes=float('inf'), 
        compare=criteria
    )
    mapocam.fit()
    solutions["mapocam_2"].append(mapocam.solutions)

    mapocam = MAPOCAM2(
        action_set_base, 
        individual, 
        clf_rf_shap, 
        outlier_detection,
        max_changes=float('inf'), 
        compare=criteria
    )
    mapocam.fit()
    solutions["mapocam_2_outlier"].append(mapocam.solutions)

    mapocam = MAPOCAM2(
        action_set_base, 
        individual, 
        clf_rf_shap, 
        outlier_detection,
        estimate_outlier=True,
        max_changes=float('inf'), 
        compare=criteria
    )
    mapocam.fit()
    solutions["mapocam_2_outlier_v2"].append(mapocam.solutions)
    

X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassif

In [51]:
for i in range(len(denied_individuals_rt)):
    sol1 = solutions["mapocam"][i]
    sol2 = solutions["mapocam_2"][i]
        
    if check_equal_solutions(sol1, sol2):
        continue

    individual = Xts.iloc[denied_individuals_rt[i]]
    table = table_with_solutions(clf_rt, individual, sol1, sol2, "mapocam1", "mapocam2", False)
    print(table)
    print("---")


                                  original  mapocam1 (0)  mapocam2 (0)
LoanDuration                     30.000000      7.000000      6.000000
CriticalAccountOrLoansElsewhere   0.000000      0.000000      1.000000
prob                              0.442441      0.625996      0.752257
---
                                  original  mapocam1 (0)  mapocam2 (0)
LoanDuration                     48.000000      7.000000      6.000000
CriticalAccountOrLoansElsewhere   0.000000      0.000000      1.000000
prob                              0.376995      0.551856      0.669958
---
                                 original  mapocam1 (0)  mapocam2 (0)
LoanDuration                     18.00000      7.000000     18.000000
CriticalAccountOrLoansElsewhere   0.00000      0.000000      1.000000
prob                              0.43974      0.637372      0.628395
---
                                  original  mapocam1 (0)  mapocam2 (0)
LoanDuration                     18.000000     34.000000     18.00000

In [52]:
for i in range(len(denied_individuals_rt)):
    sol1 = solutions["mapocam_2"][i]
    sol2 = solutions["mapocam_2_outlier"][i]
        
    if check_equal_solutions(sol1, sol2):
        continue

    individual = Xts.iloc[denied_individuals_rt[i]]
    table = table_with_solutions(clf_rt, individual, sol1, sol2, "mapocam2", "mapocam2_outlier", False)
    print(table)
    print("---")


                    original  mapocam2 (0)  mapocam2_outlier (0)
LoanDuration       12.000000      6.000000              6.000000
LoanAmount        409.000000    425.000000            425.000000
NoCurrentLoan       1.000000      1.000000              0.000000
OtherLoansAtBank    0.000000      1.000000              0.000000
prob                0.366442      0.584438              0.776332
---
                                    original  mapocam2 (0)  \
LoanDuration                       42.000000     60.000000   
LoanAmount                       9283.000000  10057.000000   
LoanRateAsPercentOfIncome           1.000000      1.000000   
SavingsAccountBalance_geq_500       0.000000      1.000000   
NoCurrentLoan                       1.000000      1.000000   
CriticalAccountOrLoansElsewhere     0.000000      1.000000   
OtherLoansAtBank                    1.000000      1.000000   
prob                                0.231438      0.534658   

                                 mapocam2_outli

In [53]:
for i in range(len(denied_individuals_rt)):
    sol1 = solutions["mapocam_2_outlier"][i]
    sol2 = solutions["mapocam_2_outlier_v2"][i]
        
    if check_equal_solutions(sol1, sol2):
        continue

    individual = Xts.iloc[denied_individuals_rt[i]]
    table = table_with_solutions(clf_rt, individual, sol1, sol2, "mapocam_2_outlier", "mapocam_2_outlier_v2", False)
    print(table)
    print("---")
